In [1]:
#instalar librerias 
#----------------------------
!pip install selenium
!pip install webdriver_manager 
!pip install datetime
!pip install undetected_chromedriver

In [2]:
import pandas as pd # libreria para trabajar con dataframes 
from selenium import webdriver # importar el driver 
from time import sleep # para generar el tiempo de espera entre busquedas 
import lxml
from selenium.webdriver.common.keys import Keys # insertar palabras en las busquedas 
from webdriver_manager.chrome import ChromeDriverManager #importar el instalador del drive 
from datetime import datetime # para detectar la fecha 
from selenium.webdriver.chrome.options import Options #agrega las opciones de navegacion en un driver 
import random # para generar tiempos de espera aleatorios 
import undetected_chromedriver.v2 as uc

In [3]:
#Instalar o carga el driver sin tener el ejecutable en la PC
options = Options()#opciones del buscador 
options.add_argument("--incognito")#ventana incognito
options.add_argument("--start-maximized")# inicializar la ventana maximizada 
browser=webdriver.Chrome(ChromeDriverManager().install(),options=options)# abrir el driver con las opciones descritas previamente 
browser.implicitly_wait(4) #cuando hay algun error para seguir probando y de no solucionarse salta al siguiente comando 

C:\Users\aldob\AppData\Local\Temp/ipykernel_4136/1306580639.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser=webdriver.Chrome(ChromeDriverManager().install(),options=options)# abrir el driver con las opciones descritas previamente


In [4]:
print("Insertar palabra") 
palabra=input() # insertar la palabra de busqueda 
correo= input("Ingrese el correo para recibir el reporte: ")

#FUNCIÓN DE BUSQUEDA EN TOTTUS ------
def tottus(palabra):
    browser.get('https://www.tottus.com.pe') #conexión con la página de Tottus 
    sleep(1) # tiempo de espera de un segundo para cargar la página 
    c_busqueda=browser.find_element("xpath","/html/body/div[1]/div[2]/div[2]/div[1]/div[1]/section/div[1]/div/input") # se busca el casillero para ingresar la palabra a buscar en el HTML
    c_busqueda.send_keys(palabra) # se coloca la palabra de busqueda (se ingresa el producto)
    sleep(1) # tiempo de espera cargar la página 
    browser.find_element("xpath","/html/body/div[1]/div[2]/div[2]/div[1]/div[1]/section/div[1]/div/button").click() # se busca la imagen de lupa para darle click y hacer la busqueda 
    sleep(1)  # tiempo de espera para cargar la página 
    lista_prod=[] # se decalara una lista vacia para guardar el nombre de todos los prooductos extraidos 
    while(1):# se genera un bucle para ir de página en página, si se encuentran varios artículos 
        try: # para se ejecute cuando no hay ningun incoveniente con la página, cuando no hay errores 
            pagina=browser.find_element("xpath",".//ul[@class='jsx-1957748114']")# se busca al contenedor de todos los productos por página 
            productos=pagina.find_elements("xpath",".//li") # se busca a los contenedores de cada producto, donde esta el precio, el nombre, la imagen y otros detalles.
            for producto in productos:# se genera un bucle para extraer la información por producto 
                info=[]# se declara una lista vacia para almacenar la información por producto 
                
                #salen productos sugeridos en la busqueda que no tienen la palabra buscada en la cadena para ello 
                #se hace un validador de busqueda
                if not palabra.lower() in producto.find_element("xpath",".//h2").text.lower(): # se busca la palabra en la cadena extraida para nombre.
                    continue # de no encontrarse la palabra, se continua con el siguiente producto 
                info.append("Tottus") # se adiciona el nombre del supermercado 
                info.append(producto.find_element("xpath",".//h2").text)# se busca el nombre del producto y se adiciona a la lista qeu almacena la información por producto 
                try: # si se encuentra el precio del producto que se adicione a la lista de detalles del producto 
                    info.append(float(producto.find_element("xpath",".//span[@class='list price medium cmrPrice ']").text.split(" ")[1]))
                    #info.append(producto.find_element("xpath",".//span[@class='list price medium cmrPrice ']").text.split(" ")[2])
                except:
                    continue
                lista_prod.append(info)# adicionar la lista de información del producto 
                print(info)
            browser.find_element("xpath",".//li[@class='next']").click()# avanzar a la siguiente página de productos con un click automatico 
        except: # si no se encuentra el boton para pasar a la siguiente página que se termine el bucle 
            break 
    return(lista_prod) # se retorna la tabla que contiene a todos los productos de tottus 
#tottus(palabra)

#FUNCIÓN DE BUSQUEDA EN METRO ------
def metro(palabra):
    browser.get("https://www.metro.pe/") # se ingresa a la página de metro 
    sleep(1) # tiempo de espera para cargar la página 
    c_busqueda=browser.find_element("xpath","/html/body/header[1]/div/div[2]/div[2]/div[1]/div[1]/input") # se busca el casillero para colocar la palabra de busqueda  
    c_busqueda.send_keys(palabra) # se ingresa a palabra a buscar 
    c_busqueda.send_keys(u'\ue007') # se coloca un enter automático para hacer la busqueda de la palabra 
    sleep(1)# tiempo de espera para cargar la página
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);") # se realiza un scroll que lleva hasta el ultimo producto de la páginapara que carguen los productos 
    sleep(7)# tiempo de espera para cargar la página después del scroll 
    pagina=browser.find_element("xpath",".//div[@class='product-shelf']") # Se extrae el contenedor de todos los productos 
    productos=pagina.find_elements("xpath",".//li[@layout='19ccd66b-b568-43cb-a106-b52f9796f5cd']") # se extrae todos los contenedores que posee los detalles de cada producto
    lista_prod=[] # lista vacia para almacenar todos los productos 
    for producto in productos: # bucle para extraer la información por producto 
        info=[]# lista para almacenar la información por producto 
        if not palabra.lower() in producto.find_element("xpath",".//a[@class='product-item__name']").text.lower(): # se busca validar  que se encuentre la palabra colocada en la cadena extraida
             continue # de no enconotrarse, que se pase a la siguiente iteración o producto
        info.append("Metro") # agregar el supermercado 
        info.append(producto.find_element("xpath",".//a[@class='product-item__name']").text) # se extrajo el nombre del producto y se adiciona a la lista de detalles 
        try: #Si se encuentra el precio que se adicione a la lista de detalles 
            info.append(float(producto.find_element("xpath",".//span[@class='product-prices__value product-prices__value--best-price']").text.split(" ")[1]))
        except: # de no entrarse que se pase al siguiente producto 
            continue
        lista_prod.append(info)#se adiciona la información de los productos a la lista general 
        print(info)
    return(lista_prod)# se retorna la lista de productos y su detalle de cada supermercado 
#metro(palabra)

#FUNCIÓN DE BUSQUEDA EN PLAZA VEA  ------
def plazaVea(palabra):
    browser.get('https://www.plazavea.com.pe/') # se ingresa a la página de plaza vea 
    sleep(1)# tiempo de espera para cargar la página 
    c_busqueda=browser.find_element("xpath","/html/body/div[3]/header/div[2]/div/div[4]/section/div[2]/div/div/input") # se busca el casillero para colocar la palabra de busqueda 
    c_busqueda.send_keys(palabra) # se inserta el producto buscado 
    c_busqueda.send_keys(u'\ue007') # se ingresa un enter automático para realizar la busqueda 
    sleep(1)# tiempo de espera para cargar la página 
    lista_prod=[]  # lista vacia para almacenar todos los productos 
    while(1):# se crea el bucle para ir de página en página de busqueda 
        try:#se ejecuta en caso no vayan a ver errores 
            pagina=browser.find_element("xpath",".//div[@class='showcase-grid ']") # se busca al contenedor de todos los aproductos 
            productos=pagina.find_elements("xpath",".//div[@class='Showcase__details']") # se extrae todos los contenedores que tienen la información por producto 
            #print(len(productos)) # se imprime la cantidad de productos encontrados 
            for producto in productos: # se busca de producto en producto la información como el precio y el nombre 
                info=[]# lista para almacenar la información por producto 
                try: # se ejecuta cuando no hay errores 
                    if not palabra.lower() in producto.find_element("xpath",".//a[@class='Showcase__name']").text.lower():# se busca l apalabra en la cadena extraida 
                        continue # al no encontrarse se pasa al siguiente contenedor de información 
                    info.append("Plaza Vea")# se soloca el supermercado 
                    info.append(producto.find_element("xpath",".//a[@class='Showcase__name']").text) # se extrae el nombre del producto 
                    info.append(float(producto.find_element("xpath",".//div[@class='Showcase__salePrice']").text.split(" ")[1]))# se extrae el precio del producto 
                except:
                    continue # que pase al siguiente contenedor en caso de haber error  
                lista_prod.append(info) # se agrega a la lista general de productos 
                print(info)
            browser.find_element("xpath",".//span[@class='pagination__item page-control next ']").click # se busca el boton para acceder a la siguiente página y se le da click
        except: #en caso de no encontrarse el boton de siguiente página se corta el bucle iniciado para cada página 
            break 
    return(lista_prod)# se retorna la lista de productos 

lista_tottus= tottus(palabra) # se ejecuta la función de busqueda en el supermercado tottus 
lista_metro= metro(palabra)  # se ejecuta la función de busqueda en el supermercado metro
lista_plaza=plazaVea(palabra) # se ejecuta la función de busqueda en el supermercado plaza Vea 


Insertar palabra
avena
Ingrese el correo para recibir el reporte: pedroyacila1199@gmail.com
['Metro', 'Avena Natural Santa Catalina 900g', 11.99]
['Metro', 'Avena con Algarrobina Santa Catalina 280g', 5.39]
['Metro', 'Avena Alpina Deslactosada Vaso 250 g', 3.5]
['Metro', 'Avena Reforzada Quinua Santa Catalina 280g', 5.39]
['Metro', 'Avena Alpina Canela Vaso 250 g', 3.5]
['Metro', 'Avena Alpina Original Vaso 250 g', 3.5]
['Metro', 'Kiwicha y Avena Reforzada Santa Catalina 280g', 5.39]
['Metro', 'Avena Precocida Santa Catalina Bolsa 500 g', 6.29]
['Metro', 'Maca y Avena Reforzada Santa Catalina 280g', 5.39]
['Metro', 'Avena Precocida Santa Catalina Bolsa 280 g', 3.99]
['Plaza Vea', 'Avena Clásica 3 OSITOS Premium Bolsa 900g', 11.9]
['Plaza Vea', 'Avena QUAKER Tradicional Bolsa 900g', 11.99]
['Plaza Vea', 'Quinua Avena 3 OSITOS Bolsa 900g', 10.9]
['Plaza Vea', 'Avena SANTA CATALINA Bolsa 1Kg', 11.99]
['Plaza Vea', 'Avena Tradicional QUAKER Bolsa 500g', 7.3]
['Plaza Vea', 'Kiwicha Avena SA

In [5]:
#print(lista_tottus)
#print(lista_metro)
#print(lista_plaza)

[]
[['Metro', 'Avena Natural Santa Catalina 900g', 11.99], ['Metro', 'Avena con Algarrobina Santa Catalina 280g', 5.39], ['Metro', 'Avena Alpina Deslactosada Vaso 250 g', 3.5], ['Metro', 'Avena Reforzada Quinua Santa Catalina 280g', 5.39], ['Metro', 'Avena Alpina Canela Vaso 250 g', 3.5], ['Metro', 'Avena Alpina Original Vaso 250 g', 3.5], ['Metro', 'Kiwicha y Avena Reforzada Santa Catalina 280g', 5.39], ['Metro', 'Avena Precocida Santa Catalina Bolsa 500 g', 6.29], ['Metro', 'Maca y Avena Reforzada Santa Catalina 280g', 5.39], ['Metro', 'Avena Precocida Santa Catalina Bolsa 280 g', 3.99]]
[['Plaza Vea', 'Avena Clásica 3 OSITOS Premium Bolsa 900g', 11.9], ['Plaza Vea', 'Avena QUAKER Tradicional Bolsa 900g', 11.99], ['Plaza Vea', 'Quinua Avena 3 OSITOS Bolsa 900g', 10.9], ['Plaza Vea', 'Avena SANTA CATALINA Bolsa 1Kg', 11.99], ['Plaza Vea', 'Avena Tradicional QUAKER Bolsa 500g', 7.3], ['Plaza Vea', 'Kiwicha Avena SANTA CATALINA Bolsa 280g', 4.5], ['Plaza Vea', "Avena Instantánea Regular

In [6]:
# se convierte en data frame la tabla de precios y productos de cada supermercado 
writer = pd.ExcelWriter('Productos_precio.xlsx', engine='openpyxl') 
a=pd.DataFrame(lista_metro, columns = ['Supermercado', 'Producto', 'Precio'])
b=pd.DataFrame(lista_tottus, columns = ['Supermercado', 'Producto', 'Precio'])
c=pd.DataFrame(lista_plaza, columns = ['Supermercado', 'Producto', 'Precio'])
frames = [a, b, c]
DF=pd.concat(frames) # se concatena las tablas 
DF=DF.sort_values('Precio')
DF.head(6)
DF.to_excel(writer,sheet_name=palabra)# convertir en un excel la tabla de productos 
writer.save()# para guardar el contenido del excel 

In [8]:
def envio(correo):
    # Colocan las opciones del navegador 
    chrome_options = uc.ChromeOptions()
    chrome_options.add_argument("--disable-extensions") # desabilitar las extenciones del driver 
    chrome_options.add_argument("--disable-popup-blocking") # desabilitar lel bloqueo de cuentas del driver 
    #opciones que permiten entrar a los correos sin problemas 
    chrome_options.add_argument("--profile-directory=Default")
    chrome_options.add_argument("--disable-plugins-discovery")
    chrome_options.add_argument("--incognito") # vetanta de incognito
    chrome_options.add_argument("user_agent=DN")
    browser = uc.Chrome(options=chrome_options)
    browser.delete_all_cookies() # eliminando los cookies del driver 
    browser.implicitly_wait(4) # tiempo de reintento por error 
    browser.get('https://accounts.google.com/AccountChooser/identifier?service=mail&continue=https%3A%2F%2Fmail.google.com%2Fmail%2F&flowName=GlifWebSignIn&flowEntry=AccountChooser') # entrar a la pestaña para colocar correos 
    cuadro= browser.find_element("xpath",'/html/body/div[1]/div[1]/div[2]/div/div[2]/div/div/div[2]/div/div[1]/div/form/span/section/div/div/div[1]/div/div[1]/div/div[1]/input') #se busaca el recuadro para colocar el correo 
    user='davidyacila22@gmail.com' # se almacena el correo de donde se haran los envios de mensajes 
    password='osonapo123456' # la contraseña 
    cuadro.send_keys(user) # se inserta el correo 
    sleep(1) # tiempo de espera para cargar la página 
    c_pass=browser.find_element("xpath",'/html/body/div[1]/div[1]/div[2]/div/div[2]/div/div/div[2]/div/div[2]/div/div[1]/div/div/button/span').click() # darle click a la opción de siguiente
    sleep(1)# tiempo de espera para cargar la página 
    cuadro=  browser.find_element("xpath",'/html/body/div[1]/div[1]/div[2]/div/div[2]/div/div/div[2]/div/div[1]/div/form/span/section/div/div/div[1]/div[1]/div/div/div/div/div[1]/div/div[1]/input')# insertar la contraseña 
    cuadro.send_keys(password) # colocar el password 
    sleep(1)# tiempo de espera para cargar la página 
    browser.find_element("xpath",'/html/body/div[1]/div[1]/div[2]/div/div[2]/div/div/div[2]/div/div[2]/div/div[1]/div/div/button/span').click() # darle click para insertar la contraseña 
    sleep(1)# tiempo de espera para cargar la página 
    browser.find_element("xpath",'/html/body/div[7]/div[3]/div/div[2]/div[1]/div[1]/div/div').click()# dar click en "redactar" para colocar un correo 
    sleep(1)# tiempo de espera para cargar la página 
    browser.find_element("xpath",".//input[@class='agP aFw']").send_keys(correo) # ingresar el correo del destinatario 
    browser.find_element("xpath",".//input[@class='aoT']").send_keys("Resumen de precios de productos") # ingresar el asunto 
    browser.close()# cerrar el driver
    browser.quit()# quitar el driver 